In [1]:
import pandas as pd
import numpy as np
import csv
import itertools
import nltk


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YMS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
vocabulary_size=8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

df = pd.read_csv("wine_data.csv")
df = df["description"]

In [3]:
train_string=df.str.cat()

train_string=train_string.lower()

In [4]:
# Split full comment into sentences
sentences=nltk.sent_tokenize(train_string)[:1000]

# Append SENTENCE_START and SENTENCE_END
sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]

print ("Parsed %d sentences." % (len(sentences)))

Parsed 1000 sentences.


In [5]:
# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))

Found 3563 unique words tokens.


In [6]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

print ("Using vocabulary size %d." % vocabulary_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))

Using vocabulary size 8000.
The least frequent word in our vocabulary is 'acidity.crisp' and appeared 1 times.


In [9]:

# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

print ("\nExample sentence: '%s'" % sentences[0])
print ("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])

#One-hot vectorized
#ind_sent=np.append([[word_to_index[w] for w in sent] for sent in tokenized_sentences],[])
X_tr=[]
for sent in tokenized_sentences:
	X_tr=X_tr+[word_to_index[w] for w in sent] 
print (X_tr[:1000])


Example sentence: 'SENTENCE_START this tremendous 100% varietal wine hails from oakville and was aged over three years in oak. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'this', 'tremendous', '100', '%', 'varietal', 'wine', 'hails', 'from', 'oakville', 'and', 'was', 'aged', 'over', 'three', 'years', 'in', 'oak', '.', 'SENTENCE_END']'
[2, 9, 552, 292, 30, 349, 12, 1543, 19, 1544, 1, 319, 256, 224, 662, 168, 13, 40, 4, 3, 2, 53, 553, 20, 1, 7, 554, 92, 6, 387, 1545, 5, 17, 0, 320, 32, 121, 0, 225, 25, 1, 7, 321, 350, 1073, 13, 5, 555, 4, 3, 2, 90, 1, 1074, 19, 427, 15, 37, 0, 11, 34, 168, 1546, 6, 11, 15, 351, 481, 1547, 4, 3, 2, 226, 1548, 22, 6, 663, 0, 49, 1, 139, 28, 826, 1, 1549, 32, 7, 1550, 6, 272, 72, 1, 58, 4, 3, 2, 9, 10, 63, 0, 273, 0, 201, 1, 1551, 16, 5, 17, 0, 8, 46, 14, 6, 428, 23, 43, 1, 179, 257, 4, 3, 2, 7, 180, 0, 1552, 37, 10, 429, 31, 1553, 90, 4, 3, 2, 21, 33, 1554, 1555, 1556, 5, 1557, 6, 7, 12, 322, 144, 32, 1075, 1558, 13, 9, 1076,

In [7]:
maxx = 0
for i in range(len(tokenized_sentences)):
    if len(tokenized_sentences[i]) >= maxx:
        maxx = len(tokenized_sentences[i])

In [10]:
len(X_tr)

28728

In [11]:
len(word_to_index)

3564

In [12]:
one_hot =  np.zeros([len(X_tr),len(word_to_index)])
one_hot.shape
    

(28728, 3564)

In [13]:
for i in range(len(X_tr)):
    one_hot[i,X_tr[i]] = 1

In [14]:
one_hot.shape

(28728, 3564)

In [23]:
empty_data = np.zeros([27709, 20, 3564])

In [26]:
L = 20
empty_data.shape

(27709, 20, 3564)

In [27]:
for i in range(empty_data.shape[0]):
    spot = -1
    for j in range(i, i+L):
        spot += 1
        empty_data[i,spot,:] = one_hot[j,:]

In [30]:
x_dat = empty_data[:,:L-1,:]
y_dat = empty_data[:,L-1,:]

In [32]:
y_dat.shape

(27709, 3564)

In [72]:
from sklearn.model_selection import train_test_split

train_size = 0.7

x_train, x_test, y_train, y_test = train_test_split(x_dat, y_dat, train_size = train_size, shuffle = False)


MemoryError: 

In [ ]:
from keras.layers.recurrent import SimpleRNN

In [73]:
maxlen = 20
model = Sequential()
model.add(SimpleRNN(n_hidden,
                   init = weight_variable,
                   input_shape = (maxlen, n_out)))
model.add(Dense(n_out, init = weight_variable))
modell.add(Activation('linear'))
optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
model-compile(loss = 'mean_squared_error',
             optimizer = optimizer)


SyntaxError: invalid syntax (<ipython-input-73-96c16a936c70>, line 11)

In [ ]:
epochs = 100
batch_size = 10 

model.fit(X_train, Y_train,
         batch_size = batch_size,
         epechs = epochs,
         validation_data = (X_validation, Y_vlaidation),
         callbacks = [early_stopping])

In [ ]:
truncate = maxlen
Z= x_dat[:1,:,:]
original = []

In [ ]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=115)
x_val = sequence.pad_sequences(x_val, maxlen=115)
x_test = sequence.pad_sequences(x_test, maxlen=115)

In [27]:
x_data = data

['SENTENCE_START',
 'juicy',
 'red-cherry',
 'fruit',
 'and',
 'a',
 'compelling',
 'hint',
 'of',
 'caramel',
 'greet',
 'the',
 'palate',
 ',',
 'framed',
 'by',
 'elegant',
 ',',
 'fine',
 'tannins',
 'and',
 'a',
 'subtle',
 'minty',
 'tone',
 'in',
 'the',
 'background',
 '.',
 'SENTENCE_END']

array([[0., 3., 0., 0., 0.]])

3564


(2, 5, 3564) (2, 1, 3564)
